In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os


In [18]:
os.curdir


'.'

In [19]:
print(os.getcwd())

/home


In [20]:
df = pd.read_csv('/music_ml/tracks_features.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/music_ml/tracks_features.csv'

In [6]:
df["month"] = df['release_date'].str.split("-").str[1]

NameError: name 'df' is not defined

In [39]:
def find_artist_song(name, song):
    str_1= "['"
    str_2 = "']"
    new = str_1 + name + str_2
    temp = df[df['artists'] == new]
    temp = temp[temp['name'].str.contains(song)]
    return temp[["year", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]]

In [40]:
df = df.dropna()
df.columns

NameError: name 'df' is not defined

In [338]:
df["year_scaled"] = df["year"] * 5

X = df[["year_scaled", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]]


In [339]:
Y = df["name"]

In [340]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [341]:
knn_model = KNeighborsClassifier(n_neighbors=5) # You can change the value of 'k' as needed.

knn_model.fit(X_train, y_train)


KNeighborsClassifier()

In [342]:
y_pred = knn_model.predict(X_test)

# calculate the accuracy of the model.

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 0.75%


In [368]:
def recommend_songs(favorite_song, favorite_artist, df, knn_model, k=5):

    #filter data
    favorite_song_info = df[(df['name'].str.contains(favorite_song)) & (df['artists'].str.contains(favorite_artist))]
    
    if favorite_song_info.empty:
        return f"No matching song '{favorite_song}' by artist '{favorite_artist}' found in the dataset."
    
    #extract the feature vector for the favorite song
    favorite_song_features = favorite_song_info[["year_scaled", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]]
    
    # Use k-NN to find similar songs
    distances, indices = knn_model.kneighbors(favorite_song_features, n_neighbors=5)

    # get the recommended songs based on indices
    recommended_songs = df.iloc[indices[0]]
    
    # filter out the favorite song 
    recommended_songs = recommended_songs[recommended_songs['name'] != favorite_song]
    
    # return the song recommendations with artist names
    recommendations = recommended_songs[['name', 'artists','year']]
    return recommendations

# example usage:
favorite_song = ""  # replace with user input
favorite_artist = "Waylon Jennings"  # replace with user input

recommendations = recommend_songs(favorite_song, favorite_artist, df, knn_model)
print(recommendations)


                                 name                 artists  year
353946                     Changeover     ['George Meanwell']  2013
500892                      Nightfall  ['Midnight Syndicate']  2008
478606                        Destiny         ['School of X']  2019
218984                Frowsey (Pt. 2)                ['Pras']  1998
946979  Escape (The Pina Colada Song)        ['The Hit Crew']  2007


In [16]:
df['year'].value_counts().sample()

NameError: name 'df' is not defined